<a href="https://colab.research.google.com/github/Jun-629/20MA573/blob/master/src/Hw10_MRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider 2-d PDE
$$\frac{1}{2} \Delta v(x) - v(x) + x_{1}^2 + x_{2}^2 - x_{1} - x_{2} - \frac{3}{2} = 0, x \in O=(0,1)^2$$
with its boundary data
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2, x \notin O.$$
The exact solution is 
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2.$$

- Idendify $\mathrm{MRP}$ with $\mathrm{CFD}$ in the form of 

\begin{align}
v(x) = \gamma \{ \ell ^h (x) + \sum_{i=1}^{d} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \} \tag{1} \\
\end{align}

__Soln:__

First, we change the form of the equation (1):

\begin{align}
v(x) = \gamma \cdot \ell ^h (x) + \gamma \cdot \sum_{i=1}^{2} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \tag{2} \\
\end{align}

Also, we already calculate that
$$\gamma = \frac{2}{2+h^{2}}, \quad \ell^{h}(x) = \frac{h^{2}}{2} [(x_{1} - \frac{1}{2})^{2} + (x_{2} - \frac{1}{2})^{2} -2], \quad p^{h}(x\pm he_{i}|x) = \frac{1}{4}.$$
Then compare (2) to the Bellman Equation:
$$v(s) = R(s) + \gamma \sum_{s' \in \mathcal S} P(s,s')v(s'),$$
we can deduce that

$$P(s,s') =
\begin{cases} 
\frac{1}{4},  & \mbox{if } ||s'-s||_1 = h \\
0, & \mbox{otherwise}
\end{cases}$$

and
$$ R(s) = \gamma \cdot \ell^h(s) = \frac{h^2}{2+h^2} [(x_{1} - \frac{1}{2})^{2} + (x_{2} - \frac{1}{2})^{2} -2] $$

- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by value iteration.
- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by Monte-Carlo method.
- For $h$ = 1/8, compute $\mathrm{CFD}$ solution by $\mathrm{TD}$ method.
- Compare above three methodsand conclude your observations.


In [0]:
# Value Iteration

import numpy as np

def gamma(N):
  h = 1/N
  gamma_value = 2/(2 + h**2)
  return gamma_value
def l(N):
  h = 1/N
  l_h = np.zeros([N+1, N+1])
  for i in range(N+1):
    for j in range(N+1):
      if i == 0 or i == N or j == 0 or j == N:
        l_h[i, j] = 0
      else:
        l_h[i, j] = h**2 / 2 *  ((i/N - 0.5)**2 + (j/N - 0.5)**2 - 2)
  return l_h
def F(N, u):
  h = 1/N
  l_h = l(N)
  v = np.zeros([N+1, N+1])
  for i in range(0, N+1):
    for j in range(0, N+1):
      if i == 0 or i == N or j == 0 or j == N:
        v[i, j] = u[i, j]
      else:
        v[i, j] = gamma(N) * l_h[i, j] + gamma(N) * (u[i+1, j] + u[i, j+1] + u[i, j-1] + u[i-1, j]) / 4
  return v
def initial_value(N):
  u = np.zeros([N+1, N+1])
  for i in range(0, N+1):
    for j in range(0, N+1):
      if i == 0 or i == N or j == 0 or j == N:
        u[i, j] = ((i/N - 0.5)**2 + (j/N - 0.5)**2)
      else:
        u[i, j] = 0
  return u
def value_iteration(N, error_hat = 0.0001):
  v = initial_value(N)
  error = 1
  step = 0
  while error > error_hat:
    step +=1
    u = v
    v = F(N, u)
    error = np.max(np.abs(u - v))
  return [error, step, v]
VI_soln = value_iteration(8)[2]
# print(VI_soln)

In [2]:
# Monte-Carlo Method
import numpy as np
import random as rd

N = 8
h = 1/N
alpha = 0.04 

s = [x for x in range(1,N)]
#rd.seed(1)
s0 = [rd.choice(s), rd.choice(s)]

S_list = [s0]
print(S_list)         # Still change S_list[0] even though set it firstly
ei = [-1,1]
T = 0                 # T is the stopping time

while s0[0]>0 and s0[0]<N and s0[1]>0 and s0[1]<N:
  s0[0] += rd.choice(ei)
  s0[1] += rd.choice(ei)
  S_list.append([s0[0],s0[1]])
  T += 1              # S_list is the state space excepet S_list[0]
print(S_list)

R = np.zeros(T+1)
v = 0
for i in range(T+1):
  x1 = S_list[i][0]
  x2 = S_list[i][1]
  R[i] = h**2/(2+h**2)*((x1/N-0.5)**2+(x2/N-0.5)**2-2)        # The reward list, (terminal is equal to 0?)
G = R[T]
V_list = [0]
for t in range(T-1,0,-1):
  G = R[t] + gamma(N) * G
  v += alpha * (G - v)
  V_list.append(v)               # Owing to the bug in S_list[0], there is no value when T=1
print(V_list)

#print(l(8)*gamma(8))   # This is the complete R(s)


[[5, 3]]
[[8, 6], [6, 4], [7, 5], [8, 6]]
[0, -0.0010909049936902828, -0.00273049235357334]


In [27]:
# TD Method

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%pylab inline
import random

# parameters
gamma = 0.5 # discounting rate
rewardSize = -1
gridSize = 8
alpha = 0.05 # (0,1] // stepSize
terminationStates = []
for i in range(gridSize):
  for j in [0,gridSize-1]:
    terminationStates.append([i,j])
for j in range(gridSize):
  for i in [0,gridSize-1]:
    terminationStates.append([i,j])
actions = [[-1, 0], [1, 0], [0, 1], [0, -1]]
numIterations = 10000

# initialization
V = np.zeros((gridSize, gridSize))
returns = {(i, j):list() for i in range(gridSize) for j in range(gridSize)}
deltas = {(i, j):list() for i in range(gridSize) for j in range(gridSize)}
states = [[i, j] for i in range(gridSize) for j in range(gridSize)]

# utils
def generateInitialState():
    initState = random.choice(states[1:-1])
    return initState

def generateNextAction():
    return random.choice(actions)

def takeAction(state, action):
    if list(state) in terminationStates:
        return 0, None
    finalState = np.array(state)+np.array(action)
    # if robot crosses wall
    if -1 in list(finalState) or gridSize in list(finalState):
        finalState = state
    return rewardSize, list(finalState)

for it in tqdm(range(numIterations)):
    state = generateInitialState()
    while True:
        action = generateNextAction()
        reward, finalState = takeAction(state, action)
        
        # we reached the end
        if finalState is None:
            break
        
        # modify Value function
        before =  V[state[0], state[1]]
        V[state[0], state[1]] += alpha*(reward + gamma*V[finalState[0], finalState[1]] - V[state[0], state[1]])
        deltas[state[0], state[1]].append(float(np.abs(before-V[state[0], state[1]])))
        
        state = finalState

V


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'gamma']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
  8%|▊         | 848/10000 [00:00<00:01, 8477.14it/s]

Populating the interactive namespace from numpy and matplotlib
[[0, 0], [0, 7], [1, 0], [1, 7], [2, 0], [2, 7], [3, 0], [3, 7], [4, 0], [4, 7], [5, 0], [5, 7], [6, 0], [6, 7], [7, 0], [7, 7], [0, 0], [7, 0], [0, 1], [7, 1], [0, 2], [7, 2], [0, 3], [7, 3], [0, 4], [7, 4], [0, 5], [7, 5], [0, 6], [7, 6], [0, 7], [7, 7]]


100%|██████████| 10000/10000 [00:01<00:00, 8667.13it/s]


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -1.41115254, -1.64865535, -1.63379813, -1.60101541,
        -1.68598132, -1.3154983 ,  0.        ],
       [ 0.        , -1.69547957, -1.89424426, -1.93153991, -1.91335127,
        -1.87959509, -1.5327981 ,  0.        ],
       [ 0.        , -1.66451303, -1.92884294, -1.98070148, -1.97699674,
        -1.92706579, -1.73318839,  0.        ],
       [ 0.        , -1.68514447, -1.91198641, -1.97410486, -1.9785462 ,
        -1.92188471, -1.72371236,  0.        ],
       [ 0.        , -1.59327308, -1.88422545, -1.91897719, -1.94123168,
        -1.89672056, -1.65726778,  0.        ],
       [ 0.        , -1.40143392, -1.70874577, -1.50436021, -1.67150033,
        -1.67256658, -1.26128555,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

0
7
